<a href="https://colab.research.google.com/github/RamBhai007/Data-Extraction-and-NLP/blob/main/blackCofferInternShip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

input_path ='/content/drive/MyDrive/BlackCofferInternShip/Input.xlsx'
stopwords_path = '/content/drive/MyDrive/BlackCofferInternShip/StopWords'
master_path = '/content/drive/MyDrive/BlackCofferInternShip/MasterDictionary'
text_path = '/content/drive/MyDrive/BlackCofferInternShip/text'
output_path='/content/drive/MyDrive/BlackCofferInternShip/Output Data Structure.xlsx'

input_df = pd.read_excel(input_path)

for index,row in input_df.iterrows():
  url_id= row['URL_ID']
  url = row['URL']

  try:
    response = requests.get(url)
  except:
    print("can't get response of {}".format(url_id))
  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(url_id))
  try:
    title = soup.find('h1').get_text()
  except:
    print("can't get title of {}".format(url_id))
    continue
  article = ""
  try:
    for p in soup.find_all('p'):
      article += p.get_text()
  except:
    print("can't get text of {}".format(url_id))

  text_name = '/content/drive/MyDrive/BlackCofferInternShip/text/' + str(url_id) + '.txt'
  with open(text_name, 'w') as file:
    file.write(title + '\n' + article)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


can't get title of 44.0
can't get title of 57.0
can't get title of 144.0


In [ ]:
stop_words = set()
pos_words  = set()
neg_words  = set()

for files in os.listdir(stopwords_path):
  with open(os.path.join(stopwords_path,files),'r',encoding='ISO-8859-1') as f:
    stop_words.update(set(f.read().splitlines()))

for files in os.listdir(master_path):
  if files =='positive-words.txt':
    with open(os.path.join(master_path,files),'r',encoding='ISO-8859-1') as f:
      pos_words.update(f.read().splitlines())
  else:
    with open(os.path.join(master_path,files),'r',encoding='ISO-8859-1') as f:
      neg_words.update(f.read().splitlines())

In [ ]:
docs=[]

for text_file in os.listdir(text_path):
  with open(os.path.join(text_path,text_file),'r') as f:
    text = f.read()
    words = word_tokenize(text)
    filtered_text = [word for word in words if word.lower() not in stop_words]
    docs.append(filtered_text)

In [ ]:
positive_words=[]
negative_words=[]
positive_Score=[]
negative_Score=[]
polarity_Score=[]
subjectivity_Score=[]

for i in range(len(docs)):
  positive_words.append([word for word in docs[i] if word.lower() in pos_words])
  negative_words.append([word for word in docs[i] if word.lower() in neg_words])
  positive_Score.append(len(positive_words[i]))
  negative_Score.append(len(negative_words[i]))
  polarity_Score.append((positive_Score[i] - negative_Score[i]) / ((positive_Score[i] + negative_Score[i]) + 0.000001))
  subjectivity_Score.append((positive_Score[i] + negative_Score[i]) / ((len(docs[i])) + 0.000001))


In [ ]:

def measure(file):
  with open(os.path.join(text_path, file),'r') as f:
    text = f.read()
    text = re.sub(r'[^\w\s.]','',text)
    sentences = text.split('.')
    num_of_sentences = len(sentences)
    words = [word  for word in text.split() if word.lower() not in stop_words ]
    num_of_words = len(words)

    complex_words=[]
    for word in words:
      vowels = 'aeiou'
      vowel_count = sum( 1 for letter in word if letter.lower() in vowels)
      if vowel_count > 2:
        complex_words.append(word)

    syllable_count = 0
    syllable_words = []

    for word in words:
      if word.endswith('es') or word.endswith('ed'):
        word = word[:-2]
      vowels = 'aeiou'
      syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
      if syllable_count_word > 2:
        syllable_words.append(word)
        syllable_count += syllable_count_word

    avg_sentence_len = num_of_words / num_of_sentences
    avg_syllable_word_count = syllable_count / len(syllable_words)
    Percent_Complex_words  =  len(complex_words) / num_of_words
    Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

    return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words),avg_syllable_word_count


In [ ]:
avg_sentence_length = []
Percentage_of_Complex_words  =  []
Fog_Index = []
complex_word_count =  []
avg_syllable_word_count =[]

for file in os.listdir(text_path):
  avglen,com,fog,len_com,avgcount = measure(file)
  avg_sentence_length.append(avglen)
  Percentage_of_Complex_words.append(com)
  Fog_Index.append(fog)
  complex_word_count.append(len_com)
  avg_syllable_word_count.append(avgcount)


In [ ]:
def cleaned_words(file):
  with open(os.path.join(text_path,file), 'r') as f:
    text = f.read()
    text = re.sub(r'[^\w\s]', '' , text)
    words = [word  for word in text.split() if word.lower() not in stop_words]
    length = sum(len(word) for word in words)
    average_word_length = length / len(words)
  return len(words),average_word_length

In [ ]:
word_count = []
average_word_length = []
for file in os.listdir(text_path):
  word,wordlen = cleaned_words(file)
  word_count.append(word)
  average_word_length.append(wordlen)


In [ ]:
def count_personal_pronouns(file):
  with open(os.path.join(text_path,file), 'r') as f:
    text = f.read()
    personal_pronouns = ["I", "we", "my", "ours", "us", "he", "she"]
    count = 0
    for pronoun in personal_pronouns:
      count += len(re.findall(r"\b" + pronoun + r"\b", text)) 
  return count

In [ ]:
pp_count = []
for file in os.listdir(text_path):
  x = count_personal_pronouns(file)
  pp_count.append(x)

In [ ]:
variables = [positive_Score,negative_Score,polarity_Score,subjectivity_Score,avg_sentence_length,
Percentage_of_Complex_words,Fog_Index,avg_sentence_length,complex_word_count,
word_count,avg_syllable_word_count,pp_count,average_word_length]

output_df=pd.read_excel(output_path)
output_df.drop([44-37,57-37,144-37], axis = 0, inplace=True)

In [ ]:
for i, var in enumerate(variables):
    output_df.iloc[:,i+2] = var

output_df.to_csv('/content/drive/MyDrive/BlackCofferInternShip/output/Output_Data.csv', index=False)